In [1]:
!pip install bs4==0.0.1

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
#Import the libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re


In [3]:
#Get the webpage and parse it as html

page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")
soup = BeautifulSoup(page.content, 'html.parser')



In [40]:
#Retrieve the day names nested in body 

day_names = soup.find_all('h3', class_ = 'DailyContent--daypartName--3G5Y8') #day names are in this tag

list_of_days = []
for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)
    
list_of_days = list_of_days[0:10]
print(list_of_days)

['Thu 25 | Day', 'Thu 25 | Night', 'Fri 26 | Day', 'Fri 26 | Night', 'Sat 27 | Day', 'Sat 27 | Night', 'Sun 28 | Day', 'Sun 28 | Night', 'Mon 29 | Day', 'Mon 29 | Night']


In [30]:
#Retrieve the weather summary nested inside the body and find temperatures

day_temp = soup.find_all('div', class_ = 'DetailsSummary--temperature--3FMlw') 

temp_list = []
for summary in day_temp:
    e = summary.text              
    temp_list.append(e)

temp_string = ' '.join(temp_list)            
temp = re.findall("([0-9]\d[°])", temp_string)      
temp = ' '.join(temp).replace('°',' ').split()  
print(len(temp))      

high = temp[0::2]
low = temp[1::2]

print("The Day tempeatures are", high)
print(f'The Night temperatures are {low}')


30
The Day tempeatures are ['58', '63', '63', '61', '59', '66', '70', '70', '64', '60', '59', '58', '58', '58', '59']
The Night temperatures are ['46', '44', '45', '46', '46', '49', '49', '48', '46', '45', '44', '45', '44', '44', '44']


In [29]:
#Extracting the summary description
summary = soup.find_all('p', class_ = 'DailyContent--narrative--3AcXd')

summary_list = []
for summary in summary:
    e = summary.text              
    summary_list.append(e)

print(summary_list)



30


In [26]:
#Temp here
#Convert Fahrenheit to celcius 
temp_celcius = []
def celcius(temp):
    for each_temp in temp:
        each_temp = (int(each_temp)- 32) * (5/9)         #converting to int type as it gives an error for float
        new_temp = round(each_temp)
        temp_celcius.append(f'{new_temp} °C')
    return temp_celcius
celcius(temp)

['14 °C',
 '8 °C',
 '17 °C',
 '7 °C',
 '17 °C',
 '7 °C',
 '16 °C',
 '8 °C',
 '15 °C',
 '8 °C',
 '19 °C',
 '9 °C',
 '21 °C',
 '9 °C',
 '21 °C',
 '9 °C',
 '18 °C',
 '8 °C',
 '16 °C',
 '7 °C',
 '15 °C',
 '7 °C',
 '14 °C',
 '7 °C',
 '14 °C',
 '7 °C',
 '14 °C',
 '7 °C',
 '15 °C',
 '7 °C']

In [37]:
#creating the dataf
df = pd.DataFrame({'Day':list_of_days, 'Temperature':temp,
                   'summary':summary_list}) 
print(df)

#df.to_csv('Weather_forecast_easy.csv', index=False)

               Day Temperature  \
0     Thu 25 | Day          58   
1   Thu 25 | Night          46   
2     Fri 26 | Day          63   
3   Fri 26 | Night          44   
4     Sat 27 | Day          63   
5   Sat 27 | Night          45   
6     Sun 28 | Day          61   
7   Sun 28 | Night          46   
8     Mon 29 | Day          59   
9   Mon 29 | Night          46   
10    Tue 30 | Day          66   
11  Tue 30 | Night          49   
12    Wed 31 | Day          70   
13  Wed 31 | Night          49   
14    Thu 01 | Day          70   
15  Thu 01 | Night          48   
16    Fri 02 | Day          64   
17  Fri 02 | Night          46   
18    Sat 03 | Day          60   
19  Sat 03 | Night          45   
20    Sun 04 | Day          59   
21  Sun 04 | Night          44   
22    Mon 05 | Day          58   
23  Mon 05 | Night          45   
24    Tue 06 | Day          58   
25  Tue 06 | Night          44   
26    Wed 07 | Day          58   
27  Wed 07 | Night          44   
28    Thu 08 |

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b76008c-93f2-488a-b8d9-c179cb7406d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>